In [1]:
# For nicer printing
options(digits=2);

In [23]:
# Read in the data
survey <- read.csv("DR_Survey2_Share_Complete.csv",header=T);

data <- survey[,-c(1:7)];
data <- data[ ,-which(names(data) %in% c("State","City","Utility","ReasonPos",
                               "ReasonNeg"))];
data[1:2,]

,HomeW,HomeWYN,HomeS,HomeSYN,AwayW,AwayWNY,AwayS,AwaySNY,IncHomeS,IncHomeW,⋯,Attitude,SubN,PBC,hot,cold,ACYN,HeatYN,valid1,valid2,valid
1,64,1,NA,0,62,1,NA,0,8,8,,5.25,4,5,0,1,1,1,1,1,1
2,68,1,73,1,65,1,78,1,4,4,,6,4.5,6,1,0,1,1,1,1,1


In [24]:
#now explore variables
summary(data);

     HomeW       HomeWYN         HomeS        HomeSYN         AwayW    
 Min.   :50   Min.   :0.00   Min.   :40    Min.   :0.00   Min.   :45   
 1st Qu.:68   1st Qu.:1.00   1st Qu.:68    1st Qu.:1.00   1st Qu.:62   
 Median :70   Median :1.00   Median :72    Median :1.00   Median :67   
 Mean   :70   Mean   :0.91   Mean   :71    Mean   :0.81   Mean   :66   
 3rd Qu.:72   3rd Qu.:1.00   3rd Qu.:75    3rd Qu.:1.00   3rd Qu.:70   
 Max.   :95   Max.   :2.00   Max.   :85    Max.   :2.00   Max.   :95   
 NA's   :80                  NA's   :164                  NA's   :151  
    AwayWNY         AwayS        AwaySNY        IncHomeS     IncHomeW 
 Min.   :0.00   Min.   :40    Min.   :0.00   Min.   : 1   Min.   : 1  
 1st Qu.:1.00   1st Qu.:70    1st Qu.:1.00   1st Qu.: 3   1st Qu.: 3  
 Median :1.00   Median :75    Median :1.00   Median : 4   Median : 4  
 Mean   :0.97   Mean   :74    Mean   :0.92   Mean   :11   Mean   : 7  
 3rd Qu.:1.00   3rd Qu.:78    3rd Qu.:1.00   3rd Qu.: 6   3rd Qu.: 6 

### Interpret  basic summaries

Interpret correlations

Now, just show top correlations

In [36]:
sel <- !is.na(apply(data,1,max));
print(sum(sel));print(length(sel))
selC = apply(data[sel,],2,var)>0

[1] 356
[1] 826


In [41]:
# Define a function
hiCor <- function(x, level){
  res <- cor(x,method="spearman");
  res1 <- res; res1[res<0] <- -res[res < 0];
  for (i in 1:dim(x)[2]){
    res1[i,i] <- 0;
  }
  sel <- apply(res1,1,max,na.rm=T) > level;
  res[sel,sel];
}
hiCor(data[sel,selC],.95)

,MC1,ComfNW2,PBC2,PBC4,BillS,NBillS,BillW,NBillW,Sqft,SqftCate,MCG,ComfortW,PBC
MC1,1.0000,-0.0821,0.0337,0.0297,0.0018,-0.0066,-0.0292,-0.0409,0.1083,0.0818,0.9530,-0.1030,0.0318
ComfNW2,-0.082,1.000,-0.039,-0.055,0.078,0.091,0.120,0.109,0.049,0.052,-0.070,0.954,-0.052
PBC2,0.034,-0.039,1.000,0.869,-0.123,-0.135,-0.096,-0.090,-0.053,-0.047,0.042,-0.022,0.966
PBC4,0.030,-0.055,0.869,1.000,-0.093,-0.109,-0.073,-0.065,-0.098,-0.099,0.057,-0.039,0.964
BillS,0.0018,0.0780,-0.1232,-0.0930,1.0000,0.9630,0.6192,0.5854,0.3891,0.3697,-0.0380,0.0901,-0.1144
NBillS,-0.0066,0.0908,-0.1345,-0.1089,0.9630,1.0000,0.6081,0.5806,0.3730,0.3530,-0.0405,0.1049,-0.1289
BillW,-0.029,0.120,-0.096,-0.073,0.619,0.608,1.000,0.967,0.315,0.276,-0.038,0.130,-0.089
NBillW,-0.041,0.109,-0.090,-0.065,0.585,0.581,0.967,1.000,0.300,0.268,-0.051,0.120,-0.083
Sqft,0.108,0.049,-0.053,-0.098,0.389,0.373,0.315,0.300,1.000,0.957,0.088,0.062,-0.074
SqftCate,0.082,0.052,-0.047,-0.099,0.370,0.353,0.276,0.268,0.957,1.000,0.055,0.067,-0.071


## How to select an orthogonal subset of predictors?

Lets take a look at principal components

In [49]:

sum(prcomp(data[sel,selC], retx=F,scale=T)$sdev^2)

[1] 110

In [64]:
plot(1:sum(selC),cumsum(prcomp(data[sel,selC], retx=F,scale=T)$sdev^2)/
     sum(prcomp(data[sel,selC], retx=F,scale=T)$sdev^2),
     ylim=c(0,1),xlab="Number of coponents",
     ylab="Fraction of variance");
res<-prcomp(data[sel,selC], retx=F,scale=T)$rotation[,1:25];
resAbs <- res;
resAbs[res<0] <- -res[res<0];
z <- cumsum(prcomp(data[sel,selC], retx=F,scale=T)$sdev^2)/
     sum(prcomp(data[sel,selC], retx=F,scale=T)$sdev^2)

for (i in 1:25){
  print(z[i]);
  print(t(res[resAbs[,i]>.2,i,drop=FALSE]));
}


[1] 0.13
    Atti1 Atti2 Atti4 INT1 Attitude
PC1  -0.2  -0.2 -0.21 -0.2    -0.22
[1] 0.18
     SN1   SN2 BillS NBillS
PC2 -0.2 -0.21 -0.22  -0.24
[1] 0.23
    IncHomeS IncAwayS Trust1 Trust3 Trust4 Trust ACYN
PC3    -0.21     -0.2  -0.23  -0.23  -0.23 -0.25 0.23
[1] 0.27
    IncHomeS IncAwayS IncEmer10S IncEmer30S ACYN
PC4    -0.25    -0.25      -0.26      -0.26 0.24
[1] 0.31
    IncAutoS
PC5    -0.21
[1] 0.35
    ComfNW1 ComfNW2  PBC2  PBC4 HabitC5 ComfortW   PBC
PC6   -0.28   -0.26 -0.23 -0.23    0.22    -0.28 -0.24
[1] 0.38
    ComfNS1 ComfNS2 ComfNS3 ComfortS
PC7    0.25    0.22    0.21     0.25
[1] 0.41
    SubNorm2 SubNorm3
PC8      0.2     0.21
[1] 0.44
    ComfNW1 ComfNW2 HabitC5 BillS ComfortW
PC9    0.22    0.24    -0.2 -0.22     0.24
[1] 0.47
     HomeS AwayS  MC2  MCG valid1 valid2 valid
PC10 -0.25 -0.24 0.21 0.21   -0.3  -0.34 -0.36
[1] 0.5
     Latitude  hot  cold valid1 valid2 valid
PC11    -0.21 0.26 -0.22  -0.26  -0.25 -0.26
[1] 0.52
      MC2 ConvB2  MCG
PC12 0.21   0

plot without title

* Regress each predictor on the remaining predictors
eliminate with the highest adjR^2

In [9]:
res <- c();
vnam <- names(data);
for (i in 2:dim(data)[2]){
  fmla <- as.formula(paste(vnam[i],paste(vnam[-c(1,i)],collapse="+"),sep="~"));
  res <- rbind(res,c(i,round(summary(lm(fmla,data=data))$r.squared,2)));
}
row.names(res) <- vnam[res[,1]];
res[order(-res[,2]),];

lEXP,12.00,0.95
lREXP,13.00,0.95
lND,7.00,0.94
lNF,4.00,0.91
lNM,3.00,0.87
lNMR,6.00,0.86
lLA,8.00,0.81
lNS,2.0,0.8
lLD,9.00,0.76
lNLgn,5.00,0.72
lLOC,10.00,0.65


### Finally select the model
  Note, that exploring only the predictor space is not leading to
     multiple comparisons issue!

  * replace lSEXP by lEXP: simpler 
  * why lLOC, lLA, lLD, lND, lNlgn: keep lND as most stable, keep lLOC?

In [11]:
fmla ~ isBad ~ lNS+lLA+FIX+lLOC+lINT+lEXP

fmla ~ isBad ~ lNS + lLA + FIX + lLOC + lINT + lEXP

* We'll need to do prediction, so drop last few years

In [12]:
data1 <- data;
data1$from <- risk$from/3600/24/365.25+1970;
dataFit <- data1[data1$from<=1997,]; #80%
dataTest <- data1[data1$from>1997,]; #20%

In [13]:
mod <- glm(isBad ~ lNS+lLA+FIX+lLOC+lINT+lEXP,family=binomial,data=dataFit);
summary(mod); 


Call:
glm(formula = isBad ~ lNS + lLA + FIX + lLOC + lINT + lEXP, family = binomial, 
    data = dataFit)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-0.842  -0.225  -0.169  -0.132   3.327  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -5.79427    0.45010  -12.87  < 2e-16 ***
lNS          0.35650    0.10268    3.47  0.00052 ***
lLA          0.17910    0.04751    3.77  0.00016 ***
FIX          0.53314    0.12943    4.12  3.8e-05 ***
lLOC         0.13590    0.04948    2.75  0.00603 ** 
lINT         0.01788    0.00925    1.93  0.05322 .  
lEXP        -0.10757    0.03829   -2.81  0.00497 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3011.5  on 13480  degrees of freedom
Residual deviance: 2739.0  on 13474  degrees of freedom
AIC: 2753

Number of Fisher Scoring iterations: 7


Note AIC: 2753

threfore lINT probably not important

### Good to order by variance explained (Anova Deviance/Df)

In [14]:
anova(mod, test="Chi");

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
NULL,NA,NA,13480,3012,NA
lNS,1.0e+00,1.7e+02,1.3e+04,2.8e+03,5.3e-38
lLA,1.0e+00,6.5e+01,1.3e+04,2.8e+03,7.0e-16
FIX,1.0e+00,1.7e+01,1.3e+04,2.8e+03,3.0e-05
lLOC,1.0e+00,1.1e+01,1.3e+04,2.8e+03,1.1e-03
lINT,1.0e+00,5.5e+00,1.3e+04,2.7e+03,1.9e-02
lEXP,1.0e+00,7.7e+00,1.3e+04,2.7e+03,5.4e-03


A similar picture, lINT does not explain as much as others

In [15]:
mod <- glm(isBad ~ lNS+lND+FIX+lEXP+lINT,family=binomial,data=dataFit);
summary(mod);


Call:
glm(formula = isBad ~ lNS + lND + FIX + lEXP + lINT, family = binomial, 
    data = dataFit)

Deviance Residuals: 
   Min      1Q  Median      3Q     Max  
-0.851  -0.223  -0.170  -0.135   3.235  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept) -4.48210    0.28367  -15.80  < 2e-16 ***
lNS          0.31994    0.11021    2.90   0.0037 ** 
lND          0.36503    0.05354    6.82  9.2e-12 ***
FIX          0.52993    0.12873    4.12  3.8e-05 ***
lEXP        -0.10508    0.03799   -2.77   0.0057 ** 
lINT         0.01847    0.00941    1.96   0.0496 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 3011.5  on 13480  degrees of freedom
Residual deviance: 2750.4  on 13475  degrees of freedom
AIC: 2762

Number of Fisher Scoring iterations: 7


### Slightly higher AIC: 2762, but a simpler model
   * don't chase the best fit, as it leads to overfitting

In [16]:
anova(mod, test="Chi");#it is good to order predictors by deviance explained

,Df,Deviance,Resid. Df,Resid. Dev,Pr(>Chi)
NULL,NA,NA,13480,3012,NA
lNS,1.0e+00,1.7e+02,1.3e+04,2.8e+03,5.3e-38
lND,1.0e+00,6.4e+01,1.3e+04,2.8e+03,1.1e-15
FIX,1.0e+00,1.8e+01,1.3e+04,2.8e+03,2.6e-05
lEXP,1.0e+00,9.3e+00,1.3e+04,2.8e+03,2.4e-03
lINT,1.0e+00,3.8e+00,1.3e+04,2.8e+03,5.0e-02


* Note: use test="F" for linear models (lm)
* Note: R's anova order matters in variance explained, as
  each SS is based on the residuals from predictors going before it
* Alternative sum of squares for ANOVA are obtained via drop1
*  In this case it is based on the residuals of remaining predictors (not
  just of preceeding predictors

In [17]:
drop1(mod, test="Chi");

,Df,Deviance,AIC,LRT,Pr(>Chi)
<none>,NA,2750,2762,NA,NA
lNS,1.0e+00,2.8e+03,2.8e+03,8.6e+00,3.3e-03
lND,1.0e+00,2.8e+03,2.8e+03,4.4e+01,2.6e-11
FIX,1.0e+00,2.8e+03,2.8e+03,1.7e+01,4.1e-05
lEXP,1.0e+00,2.8e+03,2.8e+03,7.5e+00,6.1e-03
lINT,1.00,2754.22,2764.22,3.84,0.05


### Variance inflation factor
 http://en.wikipedia.org/wiki/Variance_inflation_factor

In [19]:
library(car)
vif(mod);

lNS  lND  FIX lEXP lINT 
 2.3  3.1  1.2  1.0  1.9

￼
* Is less than 5 (max is 3.1 for lND), but lower would be better

### Exercise: other diagnostics
* is model stable if some data is dropped?
* what if we change data as in GDF?
* is there a time trend 
    isBad ~ lNS+lND+FIX+lEXP+lINT+from
* independece (residuals)

Now interpret the model
* What do estimated coefficients mean?
* Increase NS from 1 to 2, but other predictors matter
* threfore pick values that are reasonable or a median

In [20]:
try <- dataFit[1:2,];
for (i in dim(dataFit)[2])
  try[,i] <- median(dataFit[,i]);
try[1,"lNS"] <- 0;
try[2,"lNS"] <- log(2);
res <- 1/(1+exp(-predict(mod,try)));
res[2]*(1-res[1])/res[1]/(1-res[2]);

2 
1.9

### The answer is how much the risk increases if number of subsystems touched is increased from 1 to 2
  * First ratio of risk then 
  * ratio of odds

# Do prediction

In [ ]:
predicted <- 1/(1+exp(-predict(mod,dataTest)));
tapply(predicted, dataTest$isBad, mean)

## Good to know
* predicted probability is almost three times higher for the MRs that break patches

### Traditional performance:

In [ ]:
for (cutof in c(.01, .015, .02, .03, .04, .045,.1)){
  res <- table(predicted>cutof, dataTest$isBad);
  type1 <- res[2,1]/(res[1,1]+res[2,1]);
  type2 <- res[1,2]/(res[1,2]+res[2,2]);
  recall <- 1 - type2;
  precision <- res[2,2]/(res[2,1]+res[2,2]);
  print (c(cutof,type1,type2,recall,precision));
}

### A single number such as ROC typically makes little sense:
* in this case the primary concern is decent recall

### Exercise:
* compare to a random predictor
* compare to a simple predictor, e.g, more than one subsystem

### Try other prediction methods

In [ ]:
library(rpart);

fmla <- isBad ~ lNS+lND+FIX+lINT+lEXP+lLA;
er <- c();
for (w in c(20,40,60,80, 100, 110)){ 
  cart.fit <- rpart(fmla,data=dataFit,method="class",weights=as.numeric(dataFit$isBad)*w+1);
  cart.pred <- predict(cart.fit,newdata=dataTest,type="class");
  res <- table(cart.pred, dataTest$isBad);
  type1 <- res[2,1]/(res[1,1]+res[2,1]);
  type2 <- res[1,2]/(res[1,2]+res[2,2]);
  recall <- 1 - type2;
  precision <- res[2,2]/(res[2,1]+res[2,2]);
  er <- rbind (er, c(w,type1,type2,recall,precision));
}

In [ ]:
### Slightly worse than the regression model